In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import  TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat

from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import Tool

In [ ]:
serper=GoogleSerperAPIWrapper(
    serper_api_key=api_key="b58a288fbb197c86cb34240b6058acb012c791fa"
)
langchain_serper=Tool(name="internet_search", func=serper.run, description="useful when you need to search the internet")
autogen_serper=LangChainToolAdapter(langchain_serper)


ValidationError: 1 validation error for GoogleSerperAPIWrapper
  Value error, Did not find serper_api_key, please add an environment variable `SERPER_API_KEY` which contains it, or pass `serper_api_key` as a named parameter. [type=value_error, input_value={'api_key': 'b58a288fbb19...34240b6058acb012c791fa'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.12/v/value_error

In [ ]:
import os
import markdown
from dotenv import load_dotenv
from IPython.display import display, Markdown
load_dotenv(override=True)
model_client = OpenAIChatCompletionClient(
        model="gemini-2.5-flash-lite", # Use gemini-1.5-flash if 2.0 is not yet in your tier
        api_key=os.getenv("GEMINI_API_KEY"),
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        model_info=ModelInfo(
            family="gemini",
            max_input_tokens=200000,
            max_output_tokens=8192,
            mode="chat",
            vision=True,
            tool_call=True,
            function_calling=True,
            audio=False,
            json_output=True,
        structured_output=True,   # Added this line to fix UserWarning
            structured_outputs=True,  # Matches future required fields
            return_logprobs=False,
            parallel_tool_calls=True
)
)

In [ ]:
prompt = """Find a one-way non-stop flight from JFK to LHR in June 2025."""

primary_agent=AssistantAgent(
    "primary",
    model_client=model_client,
    tools=[autogen_serper],
    tools=[autogen_serper],
    system_message="You are a helpful AI research assistant who looks for promising deals on flights. Incoperate any feedback you recieve",
)



In [ ]:
evaluation_agent=AssistantAgent(
    "evaluator",
    model_client=model_client,
    system_message="Provide constructive feedback.Respond with 'APPROVE' when your feedback is approved.",
)
text_termination = TextMentionTermination("APPROVE")
team = RoundRobinGroupChat([primary_agent, evaluation_agent], termination_condition=text_termination, max_turns=20)

In [ ]:
result = await team.run(task=prompt)
for message in result.messages:
    print(f"{message.source}:\n{message.content}\n\n")